# Adaptive QoS Traffic Manager
**Fairness-Based Dynamic Bandwidth Allocator for Starlink**

In [ ]:
# AdaptiveQoS: Predictive Fairness-Based Traffic Manager for Starlink

"""
This module dynamically allocates bandwidth among Starlink users based on service tier, 
latency conditions, and a predictive 24-hour demand model. It integrates an alloy-style
fairness function that boosts emergency and underserved connections without degrading 
overall throughput.
"""

import math
import numpy as np
from typing import List, Dict
from dataclasses import dataclass

@dataclass
class UserNode:
    user_id: str
    service_tier: float  # 1.0 = premium, 0.5 = standard, 0.2 = low-tier
    qos_score: float     # From past performance history
    current_latency_ms: float
    emergency_flag: bool

class AdaptiveQoSAllocator:
    def __init__(self, total_bandwidth_mbps: float):
        self.total_bw = total_bandwidth_mbps
        self.alpha = 1.2  # Emergency multiplier
        self.beta = 0.8   # Latency penalty coefficient
        self.gamma = 0.6  # QoS weight

    def _predict_demand_modifier(self, hour):
        return 1.0 + 0.4 * math.cos(2 * math.pi * (hour - 18) / 24)

    def _user_alloy_score(self, user: UserNode, hour):
        base = user.service_tier
        latency_penalty = math.exp(-self.beta * user.current_latency_ms / 100)
        qos_boost = math.exp(self.gamma * user.qos_score)
        demand_mod = self._predict_demand_modifier(hour)
        emergency = self.alpha if user.emergency_flag else 1.0
        return base * latency_penalty * qos_boost * demand_mod * emergency

    def allocate(self, users: List[UserNode], hour: int) -> Dict[str, float]:
        scores = {user.user_id: self._user_alloy_score(user, hour) for user in users}
        total_score = sum(scores.values())
        return {uid: (score / total_score) * self.total_bw for uid, score in scores.items()}

# Example simulation
if __name__ == "__main__":
    users = [
        UserNode("u1", 1.0, 0.8, 30, False),
        UserNode("u2", 0.5, 0.5, 120, True),
        UserNode("u3", 0.2, 0.2, 200, False),
        UserNode("u4", 0.5, 0.9, 40, False),
        UserNode("u5", 0.2, 0.6, 90, True)
    ]

    allocator = AdaptiveQoSAllocator(total_bandwidth_mbps=100.0)
    for hour in range(0, 24, 6):
        print(f"\nHour: {hour:02d}:00")
        allocs = allocator.allocate(users, hour)
        for uid, mbps in allocs.items():
            print(f"  {uid}: {mbps:.2f} Mbps")